<a href="https://colab.research.google.com/github/ssu-jeong/codestates_project/blob/main/RandomForestRegressor_%EC%9E%90%EA%B1%B4%EA%B1%B0%EC%9D%B4%EC%9A%A9%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### install & import

In [ ]:
# Google Colab을 사용하는 경우 해당 셀을 실행하세요
%%capture
import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install eli5
    !pip install pandas-profiling==2.*
    !pip install pdpbox
    !pip install shap

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from datetime import datetime
#Importing Models
from sklearn.ensemble import RandomForestRegressor #,BaggingRegressor,AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.compose import TransformedTargetRegressor
#importing Preprocess 
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from category_encoders import OrdinalEncoder, TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from scipy.stats import randint,uniform

#importing Evaluation Metrics
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.metrics import mean_squared_log_error
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
#드라이브에 접근할 수 있도록 아래 코드 입력
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#불러올 파일의 경로를 filename 변수에 저장
test = '/content/drive/MyDrive/bike-sharing-demand/test.csv'
train = '/content/drive/MyDrive/bike-sharing-demand/train.csv'

datetime - hourly date + timestamp 

season -  1 = spring, 2 = summer, 3 = fall, 4 = winter 

holiday - whether the day is considered a holiday

workingday - whether the day is neither a weekend nor holiday

weather - 
1: Clear, Few clouds, Partly cloudy, Partly cloudy 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds

4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 

temp - temperature in Celsius

atemp - "feels like" temperature in Celsius
humidity - relative humidity

windspeed - wind speed

casual - number of non-registered user rentals initiated

registered - number of registered user 
rentals initiated

count - number of total rentals

In [ ]:
#pandas read_csv로 불러오기

test = pd.read_csv(test)
train = pd.read_csv(train)


train

In [ ]:
test

In [ ]:
train.shape, test.shape

In [ ]:
#둘의 합이 count이므로 제외해도 되며 test셋과 컬럼수 맞출 수 있음.
train.drop(['casual', 'registered'],1,inplace=True)

### 베이스라인모델

*  target평균
*  RMSE : 180.29981403652582



In [ ]:
train.describe()[['count']]

In [ ]:
sns.displot(train['count']); #왼쪽으로 치우친 분포

In [ ]:
sns.displot(np.log1p(train['count']),kde = True)

plt.axvline(x = np.log1p(train['count'].mean()), color = 'red');
plt.axvline(x = np.log1p(train['count'].median()), color = 'yellow');

오잉??? 이거 정규화 된거 맞나용??

In [ ]:
base_predict = train['count'].mean()
base_pred = [base_predict] * len(train['count'])

base_MSE = mean_squared_error(train['count'],base_pred)

print(base_predict)
print(f'MSE : {base_MSE}')

In [ ]:
base_RMSE = np.sqrt(base_MSE)
print(f'RMSE : {base_RMSE}')

In [ ]:
base_RMSLE = np.sqrt(mean_squared_log_error(train['count'],base_pred))
print(f'RMSLE : {base_RMSLE}')

### 데이터 전처리

In [ ]:
train.info()

다행히 결측치는 없다!


In [ ]:
train.describe()

최소값 또는 최대값에 비정상적이거나 이상한 값이 없는 것 같다. -> 어떻게 알아??


In [ ]:
#from data describtion we can extract categorcal data and numerical data
categorical_cols=['season','holiday','workingday','weather']
numerical_cols=['temp','atemp','humidity','windspeed']
target='count'

### Adding timestamp data

In [ ]:
#adding month,weekday and hour data

train['datetime'] = pd.to_datetime(train['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])
train['Month']=pd.DatetimeIndex(train['datetime']).month
test['Month']=pd.DatetimeIndex(test['datetime']).month
train['Year']=pd.DatetimeIndex(train['datetime']).year
test['Year']=pd.DatetimeIndex(test['datetime']).year
train['WeekDay']=pd.DatetimeIndex(train['datetime']).weekday
test['WeekDay']=pd.DatetimeIndex(test['datetime']).weekday
train['Hour']=pd.DatetimeIndex(train['datetime']).hour
test['Hour']=pd.DatetimeIndex(test['datetime']).hour
train.drop(columns=['datetime'],axis=1,inplace=True)

In [ ]:
categorical_cols.extend(['Month','WeekDay','Hour'])

In [ ]:
#Encoding time data ordered by mean of label in each category
def encodetime(train,test,col,target):
    d3=train[[col,target]].groupby(col).mean()
    d3.sort_values(by='count',ascending=False)
    plt.scatter(x=d3.index,y=d3['count'])
    d3=d3.sort_values(by='count')
    d3['w']=np.arange(train[col].nunique())
    dic=dict(zip(d3.index,d3['w']))
    train[col]=train[col].map(dic)
    test[col]=test[col].map(dic)

In [ ]:
encodetime(train,test,'Year',target)

In [ ]:
encodetime(train,test,'Month',target)

In [ ]:
encodetime(train,test,'Hour',target)

In [ ]:
encodetime(train,test,'WeekDay',target)

### visualizing Catergorical columns

In [ ]:
def boxplot(x,y,**kwargs):
    sns.boxplot(x=x,y=y)
    x=plt.xticks(rotation=90)
f=pd.melt(train,id_vars=['count'],value_vars=categorical_cols)
g=sns.FacetGrid(f,col='variable',col_wrap=2,sharex=False)
g.map(boxplot,'value','count')

### visualizing numiric coumns

In [ ]:
sns.pairplot(train[[*numerical_cols,'count']])

In [ ]:
f, ax = plt.subplots(figsize=(15, 15))
corr = train[[*numerical_cols,'count']].corr()
sns.heatmap(corr,cmap=sns.diverging_palette(220, 10, as_cmap=True),square=True, ax=ax, annot = True)

In [ ]:
f, ax = plt.subplots(figsize=(15, 15))
corr = train.corr()
sns.heatmap(corr,cmap=sns.diverging_palette(220, 10, as_cmap=True),square=True, ax=ax, annot = True)

### data wranging

In [ ]:
#계절과 대여관계
train[['season','count']].groupby('season').mean()

In [ ]:
sns.countplot(train['season'])

역시나 대여수는 여름가을 선선한 날이 가장 많았다.




In [ ]:
#holiday와 count
train[['holiday','count']].groupby('holiday').mean()

In [ ]:
sns.countplot(train['holiday'])

휴일인날 보다 아닌날 더 이용량이 많다. 사실 어떤게 0이고 1인지 모르겠다....그리고 엄청난 차이를 보이는 건 아니다.


In [ ]:
#workingday와 count
train[['workingday','count']].groupby('workingday').mean()

In [ ]:
sns.countplot(train['workingday'])

workingday에 이용량이 더 많다. 하지만 엄청 큰차이라고 보긴 어렵다. 

In [ ]:
# 1-> spring
# 2-> summer
# 3-> fall
# 4-> winter
sns.catplot(x='weather',data=train,kind='count')

역시나 날씨에 영향을 가장 많이 받는 것을 알 수 있다. 

In [ ]:
#corelation matrix.
cor_mat= train[:].corr()
mask = np.array(cor_mat)
mask[np.tril_indices_from(mask)] = False # to get the lower triangular shape only
fig=plt.gcf()
fig.set_size_inches(30,12)
sns.heatmap(data=cor_mat,mask=mask,square=True,annot=True,cbar=True)

### Transforming Label distribution

In [ ]:
sns.displot(train[target] , kde=True, height=8.27, aspect=11.7/8.27)
sns.displot(np.log(train[target]) , kde=True, height=8.27, aspect=11.7/8.27)

왼쪽으로 치우친 분포에서 로그 결과 사용

In [ ]:
# def trans(x,l1=0.3,l2=0):
#     if l1!=0:
#         return ((x+l2)**l1-1)/l1
#     else:
#         return np.log(x+l2)
# def rev_trans(x,l1=0.3,l2=0):
#     return (x*l1+1)**(1/l1)-l2

# z=train[target].apply(trans)   
# sns.displot(z , kde=True, height=8.27, aspect=11.7/8.27)

정규분포로 변환하는거 배우지 않았니???
그거 적용해보자!


In [ ]:
#using box cox transform on the label column

In [ ]:
train

### feature engineering 


In [ ]:
#Encoding Weather and Season (One Hot Encoding)

#train=pd.get_dummies(train,columns=['season','weather'])
#test=pd.get_dummies(test,columns=['season','weather'])

이거도 인코딩 방법 여러개 정리해서 적용해보기!


In [ ]:
#훈련/검증세트 나누기
train, val = train_test_split(train, train_size=0.8, random_state=2)
train.shape, val.shape, test.shape

In [ ]:
#데이터에서 타겟 특성 분리
target = 'count'
features = train.drop(columns=[target]).columns

In [ ]:
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [ ]:
X_train.shape, X_val.shape

In [ ]:
X_train

### Model Training and Testing

In [ ]:
#튜닝전
RFR = RandomForestRegressor()
RFR.fit(X_train,np.log(y_train))  
                       
y_pred = np.exp(RFR.predict(X_val))

In [ ]:
print('훈련 정확도', RFR.score(X_train, np.log(y_train)))
print('검증 정확도', RFR.score(X_val, np.log(y_val)))

In [ ]:
print(f'MAE_Randomforest : {mean_absolute_error(y_val,y_pred)}')
print(f'RMSE_Randomforest : {np.sqrt(mean_squared_error(y_val,y_pred))}')
# print(f'RMSLE_XGB : {np.sqrt(mean_squared_log_error(y_val,y_pred))}')

print(f'R2 : {r2_score(y_val,y_pred)}')

In [ ]:
# RandomizedSearchCV 하이퍼파라미터 설정
pipe = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestRegressor(max_depth= 20, #9
                            max_features= 9,
                        #    min_samples_leaf= 3,
                        #    min_samples_split= 19,
                           n_estimators = 376, #100
                           n_jobs=-1,  
                           random_state=2,
                           oob_score=True))

params = {'randomforestregressor__n_estimators': randint(200,400),
          'randomforestregressor__max_depth': [1,5,10,20],
        #   'randomforestregressor__min_samples_split': randint(1,20),
        #   'randomforestregressor__min_samples_leaf': randint(2,20),
          'randomforestregressor__max_features': randint(1,10)}

In [ ]:
R_CV = RandomizedSearchCV(pipe,
                          params,
                          scoring="neg_mean_squared_error",
                          n_jobs = -1, 
                          verbose=1,
                          n_iter=50,
                          cv=5)

In [ ]:
R_CV.fit(X_train,y_train)

print('최적 하이퍼파라미터:', R_CV.best_params_)
print('CV MSE: ', -R_CV.best_score_)

In [ ]:
pipe_opt = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestRegressor(max_depth= 20, #9
                           max_features= 9,
                        #    min_samples_leaf= 3,
                        #    min_samples_split= 19,
                           n_estimators = 203, #100
                           n_jobs=-1,  
                           random_state=2,
                           oob_score=True))

tt2= TransformedTargetRegressor(regressor=pipe_opt,
                               func=np.log1p,
                               inverse_func=np.expm1)

tt2.fit(X_train, y_train)
print('훈련 정확도', tt2.score(X_train, y_train))
print('검증 정확도', tt2.score(X_val, y_val))

In [ ]:
y_pred2 = tt2.predict(X_val)

print(f'MAE_Randomforest : {mean_absolute_error(y_val,y_pred2)}')
print(f'RMSE_Randomforest : {np.sqrt(mean_squared_error(y_val,y_pred2))}')
# print(f'RMSLE_XGB : {np.sqrt(mean_squared_log_error(y_val,y_pred))}')

print(f'R2 : {r2_score(y_val,y_pred2)}')

하이퍼파라미터 튜닝을 했는데 성능이 그닥...

